In [0]:
# [
#   {
#     "id": 1,
#     "name": "Alice",
#     "address": {
#       "city": "New York",
#       "zipcode": "10001"
#     },
#     "contacts": [
#       {
#         "type": "email",
#         "details": {
#           "personal": "alice@gmail.com",
#           "work": "alice@company.com"
#         }
#       },
#       {
#         "type": "phone",
#         "details": {
#           "personal": "123-456-7890",
#           "work": "987-654-3210"
#         }
#       }
#     ],
#     "skills": ["Python", "Spark", "SQL"],
#     "projects": [
#       {
#         "title": "Project A",
#         "year": 2022,
#         "tasks": [
#           {"task_name": "ETL", "status": "completed"},
#           {"task_name": "Data Cleaning", "status": "in-progress"}
#         ]
#       },
#       {
#         "title": "Project B",
#         "year": 2023,
#         "tasks": [
#           {"task_name": "ML Model", "status": "not started"}
#         ]
#       }
#     ]
#   },
#   {
#     "id": 2,
#     "name": "Bob",
#     "address": {
#       "city": "San Francisco",
#       "zipcode": "94105"
#     },
#     "contacts": [
#       {
#         "type": "email",
#         "details": {
#           "personal": "bob@gmail.com",
#           "work": "bob@company.com"
#         }
#       }
#     ],
#     "skills": ["Java", "Scala"],
#     "projects": [
#       {
#         "title": "Project X",
#         "year": 2021,
#         "tasks": [
#           {"task_name": "Data Analysis", "status": "completed"}
#         ]
#       }
#     ]
#   }
# ]

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('json').option('multiLine','true').load('/FileStore/data/complex.json')

In [0]:
df.display()

address,contacts,id,name,projects,skills
"List(New York, 10001)","List(List(List(alice@gmail.com, alice@company.com), email), List(List(123-456-7890, 987-654-3210), phone))",1,Alice,"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(Python, Spark, SQL)"
"List(San Francisco, 94105)","List(List(List(bob@gmail.com, bob@company.com), email))",2,Bob,"List(List(List(List(completed, Data Analysis)), Project X, 2021))","List(Java, Scala)"


In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



## Idea is if we have a column's datatype is array then will explode it. If it is struct the we can access it directly

##### Address

In [0]:
df.select('address.city','address.zipcode').display()

city,zipcode
New York,10001
San Francisco,94105


##### contacts

In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df_contacts = df.withColumn('new_contacts',explode(col('contacts'))).select('contacts','new_contacts')

In [0]:
df_contacts.display()

contacts,new_contacts
"List(List(List(alice@gmail.com, alice@company.com), email), List(List(123-456-7890, 987-654-3210), phone))","List(List(alice@gmail.com, alice@company.com), email)"
"List(List(List(alice@gmail.com, alice@company.com), email), List(List(123-456-7890, 987-654-3210), phone))","List(List(123-456-7890, 987-654-3210), phone)"
"List(List(List(bob@gmail.com, bob@company.com), email))","List(List(bob@gmail.com, bob@company.com), email)"


In [0]:
df_contacts.printSchema()

root
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- new_contacts: struct (nullable = true)
 |    |-- details: struct (nullable = true)
 |    |    |-- personal: string (nullable = true)
 |    |    |-- work: string (nullable = true)
 |    |-- type: string (nullable = true)



In [0]:
df_contacts.withColumn('details',df_contacts.new_contacts.details)\
    .withColumn('personal_mail',df_contacts.new_contacts.details.personal)\
    .withColumn('work_mail',df_contacts.new_contacts.details.work)\
    .withColumn('type',df_contacts.new_contacts.type)\
    .drop('contacts','new_contacts','details').display()

personal_mail,work_mail,type
alice@gmail.com,alice@company.com,email
123-456-7890,987-654-3210,phone
bob@gmail.com,bob@company.com,email


##### id, name

In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df.select('id','name').display()

id,name
1,Alice
2,Bob


##### projects

In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df.select('projects').display()

projects
"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))"
"List(List(List(List(completed, Data Analysis)), Project X, 2021))"


In [0]:
df_projects = df.withColumn('new_projects', explode(col('projects'))).select('projects','new_projects')
df_projects.display()

projects,new_projects
"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022)"
"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(List(List(not started, ML Model)), Project B, 2023)"
"List(List(List(List(completed, Data Analysis)), Project X, 2021))","List(List(List(completed, Data Analysis)), Project X, 2021)"


In [0]:
df_projects.printSchema()

root
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- new_projects: struct (nullable = true)
 |    |-- tasks: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- status: string (nullable = true)
 |    |    |    |-- task_name: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- year: long (nullable = true)



In [0]:
df_projects.select('new_projects.title','new_projects.year').display()

title,year
Project A,2022
Project B,2023
Project X,2021


In [0]:
df_tasks = df_projects.select('new_projects.tasks')

In [0]:
df_tasks_new = df_tasks.withColumn('new_tasks',explode(col('tasks')))

In [0]:
df_tasks_new.withColumn('status',df_tasks_new.new_tasks.status)\
    .withColumn('task_name',df_tasks_new.new_tasks.task_name)\
    .drop('tasks','new_tasks').display()

status,task_name
completed,ETL
in-progress,Data Cleaning
not started,ML Model
completed,Data Analysis


In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df_skills = df.withColumn('new_skills',explode('skills')).select('skills','new_skills')

In [0]:
df_skills.display()

skills,new_skills
"List(Python, Spark, SQL)",Python
"List(Python, Spark, SQL)",Spark
"List(Python, Spark, SQL)",SQL
"List(Java, Scala)",Java
"List(Java, Scala)",Scala


## Let's combine in one go

In [0]:
df.display()

address,contacts,id,name,projects,skills
"List(New York, 10001)","List(List(List(alice@gmail.com, alice@company.com), email), List(List(123-456-7890, 987-654-3210), phone))",1,Alice,"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(Python, Spark, SQL)"
"List(San Francisco, 94105)","List(List(List(bob@gmail.com, bob@company.com), email))",2,Bob,"List(List(List(List(completed, Data Analysis)), Project X, 2021))","List(Java, Scala)"


In [0]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- zipcode: string (nullable = true)
 |-- contacts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: struct (nullable = true)
 |    |    |    |-- personal: string (nullable = true)
 |    |    |    |-- work: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tasks: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- status: string (nullable = true)
 |    |    |    |    |-- task_name: string (nullable = true)
 |    |    |-- title: string (nullable = true)
 |    |    |-- year: long (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df.select('address.city','address.zipcode',explode(col('contacts')).alias('new_contacts'), 'new_contacts.details.personal'
          ,'new_contacts.details.work','new_contacts.type','id','name', 
          explode(col('projects')).alias('new_projects')).display() # multiple explode not supported in databricks. solution is to have another dataframe

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1153991937732116>:1
----> 1 df.select('address.city','address.zipcode',explode(col('contacts')).alias('new_contacts'), 'new_contacts.details.personal'
      2           ,'new_contacts.details.work','new_contacts.type','id','name', 
      3           explode(col('projects')).alias('new_projects')).display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFr

In [0]:
df_contacts_final = df.select('address.city','address.zipcode',explode(col('contacts')).alias('new_contacts'), 'new_contacts.details.personal'
          ,'new_contacts.details.work','new_contacts.type','id','name','projects','skills')

In [0]:
df_contacts_final.display()

city,zipcode,new_contacts,personal,work,type,id,name,projects,skills
New York,10001,"List(List(alice@gmail.com, alice@company.com), email)",alice@gmail.com,alice@company.com,email,1,Alice,"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(Python, Spark, SQL)"
New York,10001,"List(List(123-456-7890, 987-654-3210), phone)",123-456-7890,987-654-3210,phone,1,Alice,"List(List(List(List(completed, ETL), List(in-progress, Data Cleaning)), Project A, 2022), List(List(List(not started, ML Model)), Project B, 2023))","List(Python, Spark, SQL)"
San Francisco,94105,"List(List(bob@gmail.com, bob@company.com), email)",bob@gmail.com,bob@company.com,email,2,Bob,"List(List(List(List(completed, Data Analysis)), Project X, 2021))","List(Java, Scala)"


In [0]:
df_contacts_projects_final = df_contacts_final.select('city','zipcode','new_contacts', 'personal','work','type','id','name',explode(col('projects')).alias('new_projects'),'skills')

In [0]:
df_contacts_projects_tasks_final = df_contacts_projects_final.select('city','zipcode','new_contacts', 'personal','work','type','id','name',
                                                                     explode('new_projects.tasks').alias('new_tasks'),'new_tasks.status', 'new_tasks.task_name','skills')

In [0]:
df_contacts_projects_tasks_final.display()

city,zipcode,new_contacts,personal,work,type,id,name,new_tasks,status,task_name,skills
New York,10001,"List(List(alice@gmail.com, alice@company.com), email)",alice@gmail.com,alice@company.com,email,1,Alice,"List(completed, ETL)",completed,ETL,"List(Python, Spark, SQL)"
New York,10001,"List(List(alice@gmail.com, alice@company.com), email)",alice@gmail.com,alice@company.com,email,1,Alice,"List(in-progress, Data Cleaning)",in-progress,Data Cleaning,"List(Python, Spark, SQL)"
New York,10001,"List(List(alice@gmail.com, alice@company.com), email)",alice@gmail.com,alice@company.com,email,1,Alice,"List(not started, ML Model)",not started,ML Model,"List(Python, Spark, SQL)"
New York,10001,"List(List(123-456-7890, 987-654-3210), phone)",123-456-7890,987-654-3210,phone,1,Alice,"List(completed, ETL)",completed,ETL,"List(Python, Spark, SQL)"
New York,10001,"List(List(123-456-7890, 987-654-3210), phone)",123-456-7890,987-654-3210,phone,1,Alice,"List(in-progress, Data Cleaning)",in-progress,Data Cleaning,"List(Python, Spark, SQL)"
New York,10001,"List(List(123-456-7890, 987-654-3210), phone)",123-456-7890,987-654-3210,phone,1,Alice,"List(not started, ML Model)",not started,ML Model,"List(Python, Spark, SQL)"
San Francisco,94105,"List(List(bob@gmail.com, bob@company.com), email)",bob@gmail.com,bob@company.com,email,2,Bob,"List(completed, Data Analysis)",completed,Data Analysis,"List(Java, Scala)"


In [0]:
df_final = df_contacts_projects_tasks_final.select('city','zipcode','new_contacts', 'personal','work','type','id','name',
                                        'new_tasks','status', 'task_name',explode('skills').alias('skills')).drop('new_contacts','new_tasks')

In [0]:
df_final.display()

city,zipcode,personal,work,type,id,name,status,task_name,skills
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,completed,ETL,Python
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,completed,ETL,Spark
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,completed,ETL,SQL
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,in-progress,Data Cleaning,Python
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,in-progress,Data Cleaning,Spark
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,in-progress,Data Cleaning,SQL
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,not started,ML Model,Python
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,not started,ML Model,Spark
New York,10001,alice@gmail.com,alice@company.com,email,1,Alice,not started,ML Model,SQL
New York,10001,123-456-7890,987-654-3210,phone,1,Alice,completed,ETL,Python
